# Facial Expression Recognition (FER2013)

## Full Pipeline: Feature Extraction & GPU-Accelerated Model Benchmarking

Made By Issam Hamlil & Adiba Khattabi
It contains the complete end-to-end pipeline:
- Data loading & preprocessing
- Feature extraction (HOG, LBP, PCA, SIFT-BoVW)
- GPU-accelerated model training (cuML & XGBoost)
- Evaluation & result export


In [ ]:
import numpy as np
import cv2
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cuml
from time import time
from sklearn.model_selection import train_test_split
from skimage.feature import hog, local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# GPU-accelerated libraries
from cuml.linear_model import LogisticRegression as cuLogReg
from cuml.svm import SVC as cuSVC
from cuml.ensemble import RandomForestClassifier as cuRF
from xgboost import XGBClassifier

# --- CONFIGURATION ---
RESULTS_DIR = './FER2013_Results'
IMG_DIR = os.path.join(RESULTS_DIR, 'Confusion_Matrices')
MODELS_DIR = os.path.join(RESULTS_DIR, 'Models')
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# --- PART 1: DATA LOADING & SPLITTING ---
print("--- STEP 1: LOAD & SPLIT DATA ---")
data = np.load('processed_fer2013.npz') # Ensure this file exists in the directory
X_raw = data['X']
y_raw = data['y']

# Convert to Grayscale
X_gray = np.array([cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2GRAY) for img in X_raw])

# Encode Labels
le = LabelEncoder()
y_encoded = le.fit_transform(y_raw)
class_names = [str(c) for c in le.classes_]

# Stratified Split (80% Train, 20% Test)
X_train_img, X_test_img, y_train, y_test = train_test_split(
    X_gray, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# --- PART 2: FEATURE EXTRACTION (Step 3.2.1) ---
# 1. HOG (Histogram of Oriented Gradients)
def extract_hog(images):
    print("Extracting HOG...")
    return np.array([hog(img, orientations=9, pixels_per_cell=(8, 8),
                         cells_per_block=(2, 2)) for img in images])

# 2. LBP (Local Binary Patterns)
def extract_lbp(images, radius=3, n_points=24):
    print("Extracting LBP...")
    feats = []
    for img in images:
        lbp = local_binary_pattern(img, n_points, radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7) # Normalize
        feats.append(hist)
    return np.array(feats)

print("\n--- EXTRACTING STANDARD FEATURES ---")
X_train_hog = extract_hog(X_train_img)
X_test_hog = extract_hog(X_test_img)

X_train_lbp = extract_lbp(X_train_img)
X_test_lbp = extract_lbp(X_test_img)

# 3. PCA (Principal Component Analysis)
print("Extracting PCA...")
pca = PCA(n_components=0.95, whiten=True, random_state=42)
X_train_flat = X_train_img.reshape(X_train_img.shape[0], -1)
X_test_flat = X_test_img.reshape(X_test_img.shape[0], -1)
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)

# 4. SIFT (Scale-Invariant Feature Transform) with Bag of Words
print("Extracting SIFT Features (Bag of Words)...")
sift = cv2.SIFT_create()

def get_sift_descriptors(images):
    all_descriptors = []
    descriptors_list = []
    
    for img in images:
        kp, des = sift.detectAndCompute(img, None)
        if des is None:
            des = np.zeros((1, 128), dtype=np.float32)
        descriptors_list.append(des)
        all_descriptors.append(des)
    
    return descriptors_list, np.vstack(all_descriptors)

# Get descriptors for Vocabulary building
train_des_list, train_all_des = get_sift_descriptors(X_train_img)
test_des_list, _ = get_sift_descriptors(X_test_img)

# Build Vocabulary using K-Means
k = 100
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=1000, random_state=42, n_init=3)
kmeans.fit(train_all_des.astype(np.float32))

# Create Histograms from Vocabulary
def create_histograms(descriptor_list, model):
    histograms = []
    for des in descriptor_list:
        words = model.predict(des.astype(np.float32))
        hist, _ = np.histogram(words, bins=range(k+1))
        hist = hist.astype("float32")
        hist /= (hist.sum() + 1e-7) # Normalize
        histograms.append(hist)
    return np.array(histograms)

X_train_sift = create_histograms(train_des_list, kmeans)
X_test_sift = create_histograms(test_des_list, kmeans)

print("✅ All Features Extracted.")

# --- PART 3: MODEL TRAINING & EVALUATION (Step 3.2.2 - 3.2.4) ---
print("\n--- STARTING MODEL TOURNAMENT ---")

def get_models():
    # Note: For full compliance with Step 3.2.2, Hyperparameter tuning (GridSearch) 
    # should typically be wrapped around these models.
    return {
        "LogReg": cuLogReg(C=1.0, max_iter=1000, output_type='numpy'),
        "SVM_RBF": cuSVC(kernel='rbf', C=10, gamma='scale', cache_size=2000, output_type='numpy'),
        "RandomForest": cuRF(n_estimators=200, max_depth=25, output_type='numpy'),
        "XGBoost": XGBClassifier(n_estimators=200, learning_rate=0.1, max_depth=6, tree_method="hist", device="cuda")
    }

feature_names = ["SIFT", "HOG", "LBP", "PCA"]
scaler = StandardScaler()
results = []
y_train_gpu = y_train.astype(np.float32) # CUML models often require float32 labels

for feat in feature_names:
    print(f"\n🔹 TRAINING ON FEATURE: {feat}")
    
    # Select Feature Set
    if feat == "SIFT": X_tr, X_te = X_train_sift, X_test_sift
    elif feat == "HOG": X_tr, X_te = X_train_hog, X_test_hog
    elif feat == "LBP": X_tr, X_te = X_train_lbp, X_test_lbp
    elif feat == "PCA": X_tr, X_te = X_train_pca, X_test_pca

    # Scale Data
    X_tr = scaler.fit_transform(X_tr).astype(np.float32)
    X_te = scaler.transform(X_te).astype(np.float32)

    current_models = get_models()

    for name, model in current_models.items():
        print(f"   >> Training {name}...", end=" ")
        start = time()
        
        try:
            # Handle label types (XGBoost expects int, cuML expects float)
            y_fit = y_train if "XGBoost" in name else y_train_gpu
            
            # 1. Train
            model.fit(X_tr, y_fit)
            
            # 2. Predict
            y_pred = model.predict(X_te)
            # Convert GPU output to numpy if necessary
            if hasattr(y_pred, 'to_numpy'): y_pred = y_pred.to_numpy()
            elif hasattr(y_pred, 'get'): y_pred = y_pred.get()

            # 3. Evaluate
            acc = accuracy_score(y_test, y_pred)
            p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted', zero_division=0)
            elapsed = time() - start
            
            print(f"Acc: {acc:.2%} | F1: {f1:.2f} ({elapsed:.1f}s)")

            # 4. Save Model & Confusion Matrix
            joblib.dump(model, os.path.join(MODELS_DIR, f"{name}_{feat}.pkl"))
            
            cm = confusion_matrix(y_test, y_pred)
            plt.figure(figsize=(6, 5))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
            plt.title(f"{name} on {feat}\nAcc: {acc:.2%}, F1: {f1:.2f}")
            plt.ylabel('True')
            plt.xlabel('Predicted')
            plt.tight_layout()
            plt.savefig(os.path.join(IMG_DIR, f"CM_{name}_{feat}.png"))
            plt.close()

            results.append({
                "Feature": feat, "Model": name,
                "Accuracy": acc, "Precision": p, "Recall": r, "F1_Score": f1,
                "Time": elapsed
            })

        except Exception as e:
            print(f"❌ ERROR: {str(e)[:50]}")

# --- PART 4: SUMMARY ---
df_results = pd.DataFrame(results)
csv_path = os.path.join(RESULTS_DIR, 'FINAL_BENCHMARK_KPIs.csv')
df_results.to_csv(csv_path, index=False)
print(f"\n✅ Benchmarking Complete. Results saved to {csv_path}")